# Imports

In [2]:
import pandas as pd
import plotly.graph_objects as go
import pytz
from pathlib import Path
from plotly.subplots import  make_subplots
from datetime import timedelta as td
from datetime import datetime as dt
from hbt_tools import utils_hbt as uh

# Functions

In [ ]:
def make_df_betrieb(df_pw, on_name='On Time', off_name='Off Time'):
    # On time
    time_on = df_pw[on_name]
    time_on = time_on.sort_values()
    time_on2 = time_on - td(milliseconds=1)
    time_on = pd.DataFrame(index=time_on)
    time_on.index.rename(name='zeit', inplace=True)
    time_on['betrieb'] = 1

    time_on2 = pd.DataFrame(index=time_on2)
    time_on2.index.rename(name='zeit', inplace=True)
    time_on2['betrieb'] = 0

    # Off time
    time_off = df_pw[off_name]
    time_off = time_off.sort_values()
    time_off2 = time_off + td(milliseconds=1)
    time_off = pd.DataFrame(index=time_off)
    time_off.index.rename(name='zeit', inplace=True)
    time_off['betrieb'] = 1

    time_off2 = pd.DataFrame(index=time_off2)
    time_off2.index.rename(name='zeit', inplace=True)
    time_off2['betrieb'] = 0

    # Conatenate
    df_betrieb = pd.concat([time_on2, time_on, time_off, time_off2])
    df_betrieb.sort_index(inplace=True)

    # Compute 'betriebsdauer' / 'pumpenstunden'
    df_betrieb['betriebdauer'] = df_betrieb.index
    df_betrieb['betriebdauer'] -= df_betrieb['betriebdauer'].shift(1)

    mask = df_betrieb['betrieb']==0
    df_betrieb.loc[mask,'betriebdauer'] = td()

    df_betrieb['pumpenstunden'] = df_betrieb['betriebdauer'].cumsum()
    df_betrieb['pumpenstunden'] = df_betrieb['pumpenstunden'].dt.total_seconds() / (60**2)

    return df_betrieb
#-------------------------------------------------------------------------------

def plot_pw(df_pw, col_niveau=0, col_betrieb=4):
    fig = make_subplots(cols=1, rows=2, subplot_titles=['Niveau', 'Betrieb'],
        shared_xaxes=True)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df_pw.index, y=df_pw.iloc[:,col_niveau],
        name='Niveau'), secondary_y=False)
    
    fig.add_trace(go.Scatter(x=df_pw.index, y=df_pw.iloc[:,col_betrieb],
        name='Betrieb'), secondary_y=True)

    return fig
#-------------------------------------------------------------------------------

def make_df_pw(df_pw, A_pw, min_Q):
    # delta_h
    df_pw['delta_h'] = df_pw['wert'] - df_pw['wert'].shift()
    df_pw.iloc[0, 1] = 0

    # delta_t
    df_pw['delta_t'] = df_pw.index
    df_pw['delta_t'] -= df_pw['delta_t'].shift()
    df_pw.iloc[0, 2] = td(0)
    df_pw['delta_t'] = df_pw['delta_t'].dt.total_seconds()

    # vel_h
    df_pw['vel_h'] = df_pw['delta_h'] / df_pw['delta_t']

    min_vel = (min_Q / A_pw) * -1

    # betrieb
    df_pw['betrieb'] = 0
    mask = df_pw['vel_h'] < min_vel
    df_pw.loc[mask, 'betrieb'] = 1

    # pumpenstunden
    df_pw['pumpenstunden'] = df_pw['delta_t']
    mask = df_pw['betrieb']==0
    df_pw.loc[mask,'pumpenstunden'] = 0
    df_pw['pumpenstunden'] = df_pw['pumpenstunden'].cumsum() / (60**2)

    return df_pw

# Parameters

In [9]:
dt_format = '%d.%m.%Y %H:%M'
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Pumpenstunden")
path_hirsen = Path(r"H:\2 Projekte\8000-\8500er\8596"
    r"\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Pumpenstunden\PWHIRS 01.10.2021-31.07.2023.xlsx")
path_sonnenberg = Path(r"H:\2 Projekte\8000-\8500er\8596"
    r"\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Pumpenstunden\PWSONB 01.10.2023-31.07.2023.xlsx")

date_from = dt(2021, 9, 29, tzinfo=pytz.timezone('Europe/Zurich'))
date_to = dt(2022, 10, 1, tzinfo=pytz.timezone('Europe/Zurich'))

# Read Data

## Pumpen

In [ ]:
# PW Hirsen
id = 432
pw_hirsen = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)
pw_hirsen = pw_hirsen / 100 # cm -> m
mask = pw_hirsen.index < dt(2021, 10, 29, 12, 3, tzinfo=pytz.timezone('Europe/Zurich'))
pw_hirsen.loc[mask, 'wert'] = 0

In [ ]:
min_Q = 2e-3 # m3/s
A_pw = 5 * 4 # m2
pw_hirsen = make_df_pw(pw_hirsen, A_pw=A_pw, min_Q=min_Q)

In [ ]:
# PW Sonnenbergstrasse
id = 457
pw_sonnenberg = uh.hbtdb_get_data(id, date_from=date_from, date_to=date_to)

In [ ]:
min_Q = 2.5e-3 # m3/s
A_pw = 7.7 * 2.5 # m2
pw_sonnenberg = make_df_pw(pw_sonnenberg, A_pw=A_pw, min_Q=min_Q)

## Filterleistung

In [5]:
path_filterleistung = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung\Filterleistung\Daten-Filterleistung.xlsx")

filterleistung = pd.read_excel(path_filterleistung, sheet_name='Tabelle2',
    skiprows=1, index_col=0)

## Rain

In [ ]:
path_rain = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Regendaten\LUZ_2020 bis 2023_data.csv")

rain = uh.read_raindata_meteoschweiz(path_rain, abbr=True)
rain = rain[(rain.index>=date_from) & (rain.index<date_to)]
rain['cumsum'] = rain.cumsum()

# Computations

In [ ]:
# Betriebsstunden PW Hirsen
tot_h = pw_hirsen.iloc[-1, 5]
tot_h

In [ ]:
# Betriebsstunden PW Sonnenbergstrasse
tot_h = pw_sonnenberg.iloc[-1, 5]
tot_h

# Plots

## Plot Paul

In [ ]:
fig = make_subplots(
    specs=[[{"secondary_y": False}], [{"secondary_y": True}]],
    cols=1, rows=2,
    subplot_titles=['Filterleistung', 'Niederschlag'],
    shared_xaxes=True
    )

# Row 1
fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,0],
    name='RFB 1',
    line=dict(color='red')
), secondary_y=False, row=1, col=1)

fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,1],
    name='RFB 2',
    line=dict(color='blue')
), secondary_y=False, row=1, col=1)

val = 3
fig.add_hline(y=val, row=1, col=1)
fig.add_annotation(x=rain.index[0], y=val, text=f'{val} l/min/m2',
    font=dict(size=20, color='black'))

# Row 2
fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,0],
    name='Regen',
    line=dict(color='blue')
), secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,1],
    name='Regen kumulativ',
    line=dict(color='blue', dash='dash')
), secondary_y=True, row=2, col=1)

In [ ]:
fig.update_layout(
    font=dict(size=25)
)

fig.update_annotations(font_size=28)

save_path = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Pumpenstunden\Plot-kombiniert.html")
fig.write_html(save_path)

## Plot Filterleistung

In [13]:
fig = make_subplots()

size = 20
fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,0],
    name='RFB 1',
    marker=dict(color='red', size=size,)
))

fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,1],
    name='RFB 2',
    marker=dict(color='blue', size=size,)
))

val = 3
fig.add_hline(y=val, row=1, col=1)
fig.add_annotation(x=filterleistung.index[0], y=val, text=f'{val} l/min/m2',
    font=dict(size=25, color='black'))

fig.update_layout(
    font=dict(size=25)
)

save_path = save_dir.parent / 'Plots/Filterleistung2023.html'
fig.write_html(save_path)